### Image_Association_Prediction
In this approach, I have taken two sample images from the cropped and original. By applying TM_CCOEFF_NORMED technique , which simply slides the cropped image over the original image (as in 2D convolution) and compares the cropped and patch of original image under the cropped image

So I chose OpenCV's matchTemplate functionality for this. The matchTemplate() function is fast and easy to use, usually yields robust results. There are 6 different matching algorithms we can use with this function.

    TM_CCOEFF
    TM_CCOEFF_NORMED
    TM_CCORR
    TM_CCORR_NORMED
    TM_SQDIFF
    TM_SQDIFF_NORMED

All the _NORMED methods give output as a normalized value between 0.0 to 1.0. In TM_CCOEFF and TM_CCORR, maximum value corresponds to high probability of match, while in TM_SQDIFF_NORMED minimum value corresponds to high probability of a match. I chose TM_CCOEFF_NORMED since it gave consistent result with few of the matches I tested and it produces results in the range of 0.0 to 1.0, so that thresholding can be applied.

### Assumptions
I have made the following assumptions before proceeding the task

    1.Crops folder contains cropped patches from the Images folder
    2.Cropped images are not rotated. But they are resized while maintaining aspect ratio
    3.Duplicate images may be found in both crops and images folders
    4.Accuracy preferred over Speed
    5.Usage of OpenCV's inbuilt functions as they are more reliable androbust
    6.Sobel edge detector was not used before matching, since it didn't make any significant difference in multiple tests


### Approach

Approach I have followed:

    1.Get original images in an array
    2.Load the original image and cropped image
    3.Apply little bit of Gaussian Blur to the images to remove noise
    4.For different sizes of given template,
    5.Use cv2.matchTemplate to match the image and template
    6.Get the top-left and bottom-right corners and maximum match value
    7.Compare the max value against the last obtained max value
    8.If max value greater than last-max-value, insert maxvalue and coordinates in queue
    9.Else return the last obtained max value along with coordinates
    10.Apply thresholding, return the values if max value is greater than the threshold
    11.Append the template name and coordinates to the dictionary
    12.Repeat steps 1-5 for the all the images
    13.Finally return the dictionary containing the output


### Importing required libraries

In [ ]:
# Uncomment code to download libraries
# !pip install opencv-python
# !pip install natsort
import os
import cv2
import glob
import re
import urllib.request
import numpy as np
from natsort import natsorted,ns
import json

## Functions used in the main code
Function :	                     Description

**md5**:	                 Returns md5 checksum of the file

**getUniqueCrops** 	:       Returns hash dict of unique crops filenames

**matchCrop** 	 :            Does the template matching, returns -1 if no match found

**getCropsAssociation**   : 	Helper function to pass images and crops to matcher function

In [ ]:
numbers = re.compile(r'(\d+)')
print(numbers)
def numericalSort(value):
    parts = numbers.split(value)
    parts[1::2] = map(int, parts[1::2])
    print(parts)
    return parts

### Template Matching


Now I pass the image and template to matchTemplate() function. The matchTemplate() function takes the template and slides it over the original image pixel by pixel and calculates the match. The function returns a 2D array containing the value of matches at each position of the template over original image.
#### The cv2.matchTemplate Trick

In this case, all you need to do is apply a little trick:

    1.Loop over the input image at multiple scales (i.e. make the input image progressively smaller and smaller).
    
    2.Apply template matching using cv2.matchTemplate  and keep track of the match with the largest correlation coefficient (along with the x, y-coordinates of the region with the largest correlation coefficient).
    
    3.After looping over all scales, take the region with the largest correlation coefficient and use that as your “matched” region.


In [51]:
def matchCrop(img,crop,threshold=0.8):
    if threshold > 1.0 or threshold < 0.0:
        return -1    
    
    img = cv2.GaussianBlur(img,(5,5),0)
    crop = cv2.GaussianBlur(crop,(5,5),0)
    
    ih,iw = img.shape[:2]
    h,w = crop.shape[:2]
    
    if ih//h > 8 or iw//w > 8:
        return -1
    
    q = collections.deque([[-2,0,0],[-2,0,0]],2)

    if iw/ih > w/h :
        crop = cv2.resize(crop,(int(w*ih/h),ih))
    else:
        crop = cv2.resize(crop,(iw,int(h*iw/w)))

    for scale in range(20):

        h,w = crop.shape[:2]

        if ih//h > 8 or iw//w > 8:
            break

        res = cv2.matchTemplate(img,crop,cv2.TM_CCOEFF_NORMED)

        _, max_val, _, max_loc = cv2.minMaxLoc(res)
        bottom_right = (max_loc[0]+w , max_loc[1]+h)

        if max_val > q[-1][0]:
            if max_val > threshold:
                q.append([max_val,max_loc,bottom_right])
        else:
            break

        h = int(h*0.86)
        w = int(w*0.86)
        crop = cv2.resize(crop,(w,h))
        
    if q[-1][0] > threshold:
        return q[-1]
    else:
        return -1

In [100]:

# Returns md5 checksum of the file 
def md5(fname):
    hash_md5 = hashlib.md5()
    with open(fname, "rb") as f:
        for chunk in iter(lambda: f.read(4096), b""):
            hash_md5.update(chunk)
    return hash_md5.hexdigest()

In [53]:
#  Returns hash dict of unique crops filenames
def getUniqueCrops(crops_path):
    
    unique = {}
    for fname in glob.glob(crops_path):
        checksum = md5(fname)
        if checksum in unique:
            unique[checksum].append(fname)
        else:
            unique[checksum] = [fname]
            
    return unique
# Helper function to pass images and crops to matcher function
def getCropsAssociation(im_path,keys):
    
    matches = []    
    im = cv2.imread(im_path)

    for key in keys:
        cr = cv2.imread(unique_crops[key][0])
        result = matchCrop(im,cr)

        if result != -1:
            max_val,top_left,bottom_right = result
            
            if key not in found_crops:
                found_crops.append(key)
                
            for val in unique_crops[key]:
                crop_name = val.split('/')[1]
                matches.append((crop_name,[top_left[0],top_left[1],bottom_right[0],bottom_right[1]]))
    
    return matches


In [65]:
import hashlib
import collections
import time
import glob
import hashlib
CROPS_PATH = 'Cropped Images/'
IMAGES_PATH = 'Real Images/'

found_crops = []
not_found = []
match_dict = {}

unique_crops = getUniqueCrops(CROPS_PATH+'*')
keys = unique_crops.keys()

t1 = time.time()

images = [path.split('/')[1] for path in glob.glob(IMAGES_PATH+'*')]

for image_name in images:
    print(IMAGES_PATH+image_name)
    match_dict[image_name] = getCropsAssociation(IMAGES_PATH+image_name,keys)

for key in keys:
    if key not in found_crops:
        cr_names = []
        for path in unique_crops[key]:
            not_found.append((path.split('/')[1],[]))
            
match_dict['na'] = not_found

t2 = time.time()

print(t2-t1," secs")

Real Images/image_name_139.jpg
Real Images/image_name_101.jpg
Real Images/image_name_49.jpg
Real Images/image_name_55.jpg
Real Images/image_name_73.jpg
Real Images/image_name_3.jpg
Real Images/image_name_129.jpg
Real Images/image_name_113.jpg
Real Images/image_name_117.jpg
Real Images/image_name_53.jpg
Real Images/image_name_18.jpg
Real Images/image_name_93.jpg
Real Images/image_name_84.jpg
Real Images/image_name_122.jpg
Real Images/image_name_138.jpg
Real Images/image_name_81.jpg
Real Images/image_name_25.jpg
Real Images/image_name_26.jpg
Real Images/image_name_41.jpg
Real Images/image_name_114.jpg
Real Images/image_name_52.jpg
Real Images/image_name_56.jpg
Real Images/image_name_134.jpg
Real Images/image_name_79.jpg
Real Images/image_name_48.jpg
Real Images/image_name_103.jpg
Real Images/image_name_77.jpg
Real Images/image_name_97.jpg
Real Images/image_name_29.jpg
Real Images/image_name_80.jpg
Real Images/image_name_105.jpg
Real Images/image_name_135.jpg
Real Images/image_name_7.jpg


## Result

In [66]:
match_dict

{'image_name_139.jpg': [('crop_name_201.jpg', [0, 0, 400, 387]),
  ('crop_name_53.jpg', [0, 0, 400, 343]),
  ('crop_name_0.jpg', [0, 0, 400, 402]),
  ('crop_name_94.jpg', [0, 0, 400, 438]),
  ('crop_name_37.jpg', [0, 0, 400, 409]),
  ('crop_name_188.jpg', [0, 0, 400, 376]),
  ('crop_name_147.jpg', [0, 0, 400, 319]),
  ('crop_name_31.jpg', [0, 0, 400, 326]),
  ('crop_name_306.jpg', [4, 299, 49, 367]),
  ('crop_name_142.jpg', [0, 0, 400, 356]),
  ('crop_name_179.jpg', [0, 0, 400, 426]),
  ('crop_name_265.jpg', [0, 0, 400, 523]),
  ('crop_name_258.jpg', [0, 0, 400, 440]),
  ('crop_name_144.jpg', [0, 0, 400, 493]),
  ('crop_name_307.jpg', [0, 125, 400, 502]),
  ('crop_name_48.jpg', [0, 0, 400, 430]),
  ('crop_name_305.jpg', [0, 0, 400, 473]),
  ('crop_name_9.jpg', [0, 0, 400, 341]),
  ('crop_name_195.jpg', [0, 0, 400, 436])],
 'image_name_101.jpg': [('crop_name_201.jpg', [0, 0, 400, 387]),
  ('crop_name_53.jpg', [0, 0, 400, 343]),
  ('crop_name_0.jpg', [0, 0, 400, 402]),
  ('crop_name_94.j